# Data Preparing (Engineering)

## Récuperation des tables nécessaires

In [1]:
import pandas as pd

matchsTrain = pd.read_csv('../csv/X_Train.csv')
matchsTest = pd.read_csv('../csv/X_Test.csv')
players = pd.read_csv('../csv/Player.csv')
teams = pd.read_csv('../csv/Team.csv')
team_attr = pd.read_csv('../csv/Team_Attributes.csv')
player_attr = pd.read_csv('../csv/Player_Attributes.csv')

In [4]:
# Determine the label of the match (0: tie , 1: home team won, -1: home team lost)
def det_label(score1, score2):
    if(score1 == score2):
        return 0
    if(score1 < score2):
        return -1
    else:
        return 1

In [5]:
#On va travailler sur une copie du training set
matchsTrainCopy = matchsTrain.copy()
matchsTrainCopy['label'] = matchsTrainCopy.apply(lambda row: det_label(
            row['home_team_goal'], row['away_team_goal']), axis=1)
#Suppression des colonnes qui n'ont pas d'importances
matchsTrainCopy.drop(['country_id', 'league_id','match_api_id', 'Unnamed: 0'], axis=1, inplace=True)

## Cleaning des valeurs Nan présentent dans le training set

In [6]:
matchsTrainCopy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20479 entries, 0 to 20478
Data columns (total 76 columns):
index               20479 non-null int64
id                  20479 non-null int64
season              20479 non-null object
stage               20479 non-null int64
date                20479 non-null object
home_team_api_id    20479 non-null int64
away_team_api_id    20479 non-null int64
home_team_goal      20479 non-null int64
away_team_goal      20479 non-null int64
home_player_X1      19033 non-null float64
home_player_X2      19033 non-null float64
home_player_X3      19025 non-null float64
home_player_X4      19025 non-null float64
home_player_X5      19025 non-null float64
home_player_X6      19025 non-null float64
home_player_X7      19025 non-null float64
home_player_X8      19025 non-null float64
home_player_X9      19025 non-null float64
home_player_X10     19025 non-null float64
home_player_X11     19025 non-null float64
away_player_X1      19025 non-null float64
away

Nous pouvons voir qu'il nous manque des données seulement dans les colonnes home/away_player_i et home/away_player_XYi où 1<i<11

In [8]:
#Nous remplissons les ids des joureurs manquants par 0
matchsTrainCopy[['home_player_'+str(i) for i in range(1,12)]].fillna(0)
matchsTrainCopy[['away_player_'+str(i) for i in range(1,12)]].fillna(0)
# et tout le restant (les coordonnées des joueurs) par la valeurs qui revient le plus 
matchsTrainCopy = matchsTrainCopy.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [9]:
matchsTrainCopy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20479 entries, 0 to 20478
Data columns (total 76 columns):
index               20479 non-null int64
id                  20479 non-null int64
season              20479 non-null object
stage               20479 non-null int64
date                20479 non-null object
home_team_api_id    20479 non-null int64
away_team_api_id    20479 non-null int64
home_team_goal      20479 non-null int64
away_team_goal      20479 non-null int64
home_player_X1      20479 non-null float64
home_player_X2      20479 non-null float64
home_player_X3      20479 non-null float64
home_player_X4      20479 non-null float64
home_player_X5      20479 non-null float64
home_player_X6      20479 non-null float64
home_player_X7      20479 non-null float64
home_player_X8      20479 non-null float64
home_player_X9      20479 non-null float64
home_player_X10     20479 non-null float64
home_player_X11     20479 non-null float64
away_player_X1      20479 non-null float64
away

Il n'y a plus aucune valeur null

## Exploitation des coordonnées XY des joueurs

L'idée ici sera de transformer les coordonnées Y (nous n'allons pas utiliser les coordonnées X) en formation par exemple 442 

In [ ]:
from collections import Counter
#Fonction pour creer une formation apd des coordonnées Y des joureurs
def create_formation(row, home):
    list_form = list()  # Nous avons besoin d'une liste pour le Counter
    if(home):
        list_form = row.loc[row.index.str.startswith(
            'home_player_Y')].tolist()[1:]  # Nous ne prenons pas le gardien
    else:
        list_form = row.loc[row.index.str.startswith(
            'away_player_Y')].tolist()[1:]
    # Nous creeons un dictionnaire avec les occurences des positions des joueurs
    couter = Counter(list_form)
    couter_val = Counter(sorted(couter.elements())).values()
    # on concatene : 442
    form = ''.join((str(e) for e in list(couter_val)))
    return form

## Ajout des notes générales des joueurs

## Ajout du potentiel des joueurs

## Ajout des attributs defensePressure et le buildUpPlayPassing 